# Here is the extra recommendations build for Requirement 2
### The data understanding phase took place in collaborative.ipynb
### I still cleaned the data by deleting the director column and rows with missing cast/country

In [18]:
# Import
import pandas as pd

df_ratings = pd.read_csv('movies_ratings.csv')
df_titles = pd.read_csv('movies_titles.csv')
df_users = pd.read_csv('movies_users.csv')

In [19]:
df_titles.drop('director', axis=1, inplace=True)
df_titles.dropna(inplace=True)
df_ratings = df_ratings[df_ratings['show_id'].isin(df_titles['show_id'])]

#### Genre vectorization - this takes in similarity scores based on genre

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Set up genre matrix
# Reset index so similarity lookup matches
df_titles = df_titles.reset_index(drop=True)

# Now it's safe to use row index for similarity lookup
genre_cols = df_titles.loc[:, "Action":"Thrillers"]
genre_matrix = genre_cols.to_numpy()
similarity_matrix = cosine_similarity(genre_matrix)

# Compute cosine similarity across all titles
similarity_matrix = cosine_similarity(genre_matrix)

# Function: recommend similar movies based on title
def recommend_similar(title, top_n=10):
    target_row = df_titles[df_titles["title"] == title]
    if target_row.empty:
        return []
    idx = target_row.index[0]
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_indices = [i for i, score in sim_scores[1:top_n+1]]
    return df_titles.iloc[top_indices][["title", "show_id"]]

# Example usage
recommend_similar("The Super Mario Bros. Super Show!")

,title,show_id
40,Numberblocks,s66
56,Octonauts: Above & Beyond,s99
59,Tayo the Little Bus,s105
65,Sharkdog,s112
72,Pororo - The Little Penguin,s125
95,Kid-E-Cats,s154
155,Go! Go! Cory Carson,s255
157,Mother Goose Club,s264
162,Winx Club,s270
164,Fast & Furious Spy Racers,s273


In [21]:
def extract_genres(df):
    # Columns that are definitely not genres
    non_genre_cols = [
        'show_id', 'type', 'title', 'cast', 'country',
        'release_year', 'rating', 'duration', 'description'
    ]

    # Get all int64 columns that aren't in non-genre list
    genre_cols = [col for col in df.select_dtypes(include='int64').columns if col not in non_genre_cols]

    # Combine active genres into a string
    df['genres'] = df[genre_cols].apply(
        lambda row: ', '.join([col for col in genre_cols if row[col] == 1]), axis=1
    )

    return df

In [22]:
df_titles = extract_genres(df_titles)
df_titles.head()

,show_id,type,title,cast,country,release_year,rating,duration,description,Action,...,Musicals,Nature TV,Reality TV,Spirituality,TV Action,TV Comedies,TV Dramas,Talk Shows TV Comedies,Thrillers,genres
0,s1,Movie,Dick Johnson Is Dead,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirs...",United States,2020,PG-13,90 min,As her father nears the end of his life filmma...,0,...,0,0,0,0,0,0,0,0,0,Documentaries
1,s2,TV Show,Blood & Water,Ama Qamata Khosi Ngema Gail Mabalane Thabang M...,South Africa,2021,TV-MA,2 Seasons,After crossing paths at a party a Cape Town te...,0,...,0,0,0,0,0,0,1,0,0,"Dramas, TV Dramas"
2,s5,TV Show,Kota Factory,Mayur More Jitendra Kumar Ranjan Raj Alam Khan...,India,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,0,...,0,0,0,0,0,1,0,0,0,"Comedies, TV Comedies"
3,s8,Movie,Sankofa,Kofi Ghanaba Oyafunmike Ogunlano Alexandra Dua...,United States Ghana Burkina Faso United Kingdo...,1993,TV-MA,125 min,On a photo shoot in Ghana an American model sl...,0,...,0,0,0,0,0,0,0,0,0,Dramas
4,s9,TV Show,The Great British Baking Show,Mel Giedroyc Sue Perkins Mary Berry Paul Holly...,United Kingdom,2021,TV-14,9 Seasons,A talented batch of amateur bakers face off in...,0,...,0,0,1,0,0,0,0,0,0,Reality TV


In [23]:
recs = []

for idx, row in df_titles.iterrows():
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]  # skip itself
    for sim_idx, score in sim_scores:
        recs.append({
            "show_id": row["show_id"],
            "recommended_id": df_titles.iloc[sim_idx]["show_id"],
            "genre": df_titles.iloc[sim_idx]["genres"],
            "score": round(score, 4)
        })

df_recs = pd.DataFrame(recs)

In [24]:
import sqlite3

conn = sqlite3.connect("movie_recommendations_titles.db")

# Save titles table (optional if already in DB)
df_titles.to_sql("genres_recs", conn, if_exists="replace", index=False)

# Save the recommendations
df_recs.to_sql("genre_recommendations", conn, if_exists="replace", index=False)

76130

### Genre clustering - takes genre and RATING into consideration

In [25]:
genre_cols = df_titles.loc[:, "Action":"Thrillers"]
ratings_encoded = pd.get_dummies(df_titles["rating"], prefix="rating")

features = pd.concat([genre_cols, ratings_encoded], axis=1)
feature_matrix = features.to_numpy()

# Step 3: Compute cosine similarity between all titles
similarity_matrix = cosine_similarity(feature_matrix)

### this collects the top 10 movies in the genre

In [26]:
recs = []

for idx, row in df_titles.iterrows():
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]  # skip self
    for sim_idx, score in sim_scores:
        recs.append({
            "show_id": row["show_id"],
            "recommended_id": df_titles.iloc[sim_idx]["show_id"],
            "score": round(score, 4)
        })

df_recs = pd.DataFrame(recs)

In [27]:
# Connect to SQLite
conn = sqlite3.connect("movie_recommendations_genres.db")

# Save titles (optional if already saved elsewhere)
df_titles.to_sql("movies_titles", conn, if_exists="replace", index=False)

# Save recommendations
df_recs.to_sql("content_recommendations", conn, if_exists="replace", index=False)

conn.close()

## To be creative, we take past user ratings to recommend movies to a guest

In [63]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import pandas as pd

# Convert your ratings DataFrame to Surprise format
reader = Reader(rating_scale=(1, 5))
# Step 1: Only shows with 2+ ratings
rating_counts = df_ratings["show_id"].value_counts()
valid_ids = rating_counts[rating_counts >= 1].index

# Step 2: Filter average rating between 3 and 4
avg_ratings = df_ratings.groupby("show_id")["rating"].mean()
mid_range_ids = avg_ratings[(avg_ratings >= 3) & (avg_ratings <= 5)].index

# Step 3: Intersect both conditions
final_ids = set(valid_ids) & set(mid_range_ids)

# Step 4: Filter ratings
filtered_ratings = df_ratings[df_ratings["show_id"].isin(final_ids)]
data = Dataset.load_from_df(filtered_ratings[["user_id", "show_id", "rating"]], reader)

# Train on the whole set (since it's pre-login)
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

In [64]:
# Get all show_ids
all_movie_ids = df_titles["show_id"].unique()

# Predict ratings for a dummy user_id (e.g. 'guest')
predictions = [(movie_id, model.predict("guest", movie_id).est) for movie_id in all_movie_ids]

# Sort by estimated rating
top_recs = sorted(predictions, key=lambda x: x[1], reverse=True)[:10]

# Join with titles
top_df = pd.DataFrame(top_recs, columns=["show_id", "predicted_rating"])
top_titles = df_titles[["title", "show_id"]].merge(top_df, on="show_id")
top_titles = top_titles.sort_values(by="predicted_rating", ascending=False).reset_index(drop=True)

In [65]:
top_titles

,title,show_id,predicted_rating
0,Dark Tourist,s4760,4.171295
1,Durarara!!,s1041,4.166059
2,Naruto Shippuden: The Movie: The Lost Tower,s61,4.160925
3,Equestria Girls: Tales of Canterlot High,s5238,4.154978
4,Ex-Boyfriend,s6718,4.153765
5,DEATH NOTE,s5752,4.149572
6,A Night at the Roxbury,s6050,4.149118
7,Kuroko's Basketball,s155,4.148205
8,Tramps,s5525,4.141702
9,Challenger,s1991,4.139643


In [66]:
guest_recs = []
for mid in df_titles["show_id"].unique():
    pred = model.predict("guest", mid).est
    guest_recs.append((mid, pred))

guest_recs_df = pd.DataFrame(guest_recs, columns=["show_id", "predicted_rating"])

top_df = df_titles[df_titles["show_id"].isin(guest_recs_df.sort_values(by="predicted_rating", ascending=False).head(50)["show_id"])]
top_df = top_df.merge(guest_recs_df, on="show_id")
top_df["genre"] = "TopPicks"

genre_recs = []
for genre in ["Action", "Adventure", "Children", "Comedies", "Documentaries", "Dramas", 
              "Fantasy", "Horror Movies", "Musicals", "Reality TV", "Thrillers"]:
    genre_ids = df_titles[df_titles[genre] == 1]["show_id"]
    genre_top = guest_recs_df[guest_recs_df["show_id"].isin(genre_ids)].sort_values(by="predicted_rating", ascending=False).head(10)
    genre_df = df_titles[df_titles["show_id"].isin(genre_top["show_id"])].merge(genre_top, on="show_id")
    genre_df["genre"] = genre
    genre_recs.append(genre_df)

In [67]:
final_df = pd.concat([top_df] + genre_recs)

In [70]:
with sqlite3.connect("recommendations_collab_b4Logon.db") as conn:
    final_df[["genre", "title", "show_id", "predicted_rating"]].to_sql("Recommendations", conn, if_exists="replace", index=False)

In [72]:
import sqlite3

# Paths
source_db = "recommendations_collab_b4Logon.db"
target_db = "/Users/twoscoops/Documents/BYU/Winter25/INTEX2025/backend/INTEX2025.API/recommender.db"
table_name = "Recommendations"

# Connect to source and target
source_conn = sqlite3.connect(source_db)
target_conn = sqlite3.connect(target_db)
# Get table DDL from source
ddl = source_conn.execute(
    f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table_name}'"
).fetchone()[0]

# Create table in target (if not already exists)
target_conn.execute(f"DROP TABLE IF EXISTS {table_name}")
target_conn.execute(ddl)

# Copy data from source
data = source_conn.execute(f"SELECT * FROM {table_name}").fetchall()

# Get column count
col_count = len(source_conn.execute(f"PRAGMA table_info({table_name})").fetchall())
placeholders = ",".join(["?"] * col_count)

# Insert into target
target_conn.executemany(
    f"INSERT INTO {table_name} VALUES ({placeholders})", data
)

# Commit and close
target_conn.commit()
source_conn.close()
target_conn.close()

##### Take created databases from all these to combine into final production level one

In [ ]:
import sqlite3
import shutil
import os

# Step 1: Use one DB as the base for merging
shutil.copyfile("content_matrix_recommendation_on_show_id.db", "recommender.db")

# Step 2: Connect to the new combined database
combined_conn = sqlite3.connect("recommender.db")
combined_cursor = combined_conn.cursor()

# Step 3: Helper to attach and copy tables
def attach_and_copy_table(source_db, table_name, alias):
    print(f"Merging table '{table_name}' from '{source_db}'...")
    combined_cursor.execute(f"ATTACH DATABASE '{source_db}' AS {alias}")
    combined_cursor.execute(
        f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM {alias}.{table_name}"
    )
    combined_cursor.execute(f"DETACH DATABASE {alias}")

# Step 4: Attach and copy each source DB/table
attach_and_copy_table("collab_userId_top10.db", "user_recs", "collabdb")
attach_and_copy_table("b4logon_top10_movietitles.db", "content_recommendations", "contentdb")
attach_and_copy_table("genre_title__general_b4Logon.db", "genre_recommendations", "genredb")
attach_and_copy_table("top5_based_showId.db", "show_details", "top5db")
attach_and_copy_table("topPicksList.db", "Recommendations", "recsdb")

# Step 5: Save and close
combined_conn.commit()
combined_conn.close()

print("✅ All tables successfully merged into 'recommender.db'")

Merging table 'user_recs' from '../backend/INTEX2025.API/collab_userId_top10.db'...
Merging table 'content_recommendations' from '../backend/INTEX2025.API/b4logon_top10_movietitles.db'...
Merging table 'genre_recommendations' from '../backend/INTEX2025.API/genre_title__general_b4Logon.db'...
Merging table 'show_details' from '../backend/INTEX2025.API/top5_based_showId.db'...
Merging table 'Recommendations' from '../backend/INTEX2025.API/topPicksList.db'...
✅ All tables successfully merged into 'recommender.db'
